# 📘 04 — Model Selection

## 🧭 Tabla de Contenido
1. [Setup y Carga de Datos](#sec1)
2. [Preparación de Datos para Modelos](#sec2)
   - [División Train/Validation/Test](#sec2_1)
   - [Pipeline de Preprocesamiento](#sec2_2)
3. [Definición de Modelos Base](#sec3)
4. [Estrategia de Validación](#sec4)
5. [Entrenamiento de Modelos Base](#sec5)
6. [Comparación de Modelos](#sec6)
7. [Curvas de Aprendizaje](#sec7)
8. [Análisis de Predicciones](#sec8)
9. [Selección de Mejores Modelos](#sec9)
10. [Resumen Ejecutivo](#sec10)


# 1. Setup y Carga de Datos <a id='sec1'></a>
**Intención:** configurar el entorno, cargar datos y verificar integridad.

- Importación de librerías
- Configuración visual
- Carga de coffee_features.csv
- Revisión de integridad

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

np.random.seed(42)

df = pd.read_csv('../data/processed/coffee_features.csv')
df.head()

# 2. Preparación de Datos para Modelos <a id='sec2'></a>
**Intención:** preparar entradas y evitar leakage.

## 2.1 División Train/Validation/Test <a id='sec2_1'></a>
**Intención:** separar datos en conjuntos para evaluación justa.

In [ ]:
X = df.drop(columns=['total_cup_points'])
y = df['total_cup_points']

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.30, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.50, random_state=42)

## 2.2 Pipeline de Preprocesamiento <a id='sec2_2'></a>
**Intención:** asegurar que el preprocesamiento se aplique correctamente en train y test.

In [ ]:
from sklearn.preprocessing import StandardScaler

numeric_cols = X.select_dtypes(include=['float64','int64']).columns.tolist()

preprocessor = ColumnTransformer(
    transformers=[('num', StandardScaler(), numeric_cols)],
    remainder='passthrough'
)

# 3. Definición de Modelos Base <a id='sec3'></a>
**Intención:** evaluar modelos de distintas familias.

- Modelos lineales
- KNN
- Árboles
- Ensembles
- SVM
- Redes neuronales

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor

models = {
    'LinearRegression': LinearRegression(),
    'Ridge': Ridge(alpha=1.0),
    'Lasso': Lasso(alpha=0.001),
    'KNN': KNeighborsRegressor(n_neighbors=7),
    'RandomForest': RandomForestRegressor(n_estimators=200, random_state=42),
    'SVR': SVR(kernel='rbf', C=10),
    'MLP': MLPRegressor(hidden_layer_sizes=(128,64), max_iter=500)
}

# 4. Estrategia de Validación <a id='sec4'></a>
**Intención:** definir cómo medir rendimiento de forma robusta.
- Si usar k-fold cross validation
- ¿Metricas a usar? ¿MAE, RMSE, R2?

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# 5. Entrenamiento de Modelos Base <a id='sec5'></a>
**Intención:** evaluar cada modelo usando CV y guardar resultados.

In [ ]:
results = []

for name, model in models.items():
    pipe = Pipeline(steps=[('preprocess', preprocessor), ('model', model)])
    scores = cross_val_score(pipe, X_train, y_train, cv=kf, scoring='r2')
    results.append({'Modelo': name, 'R2_mean': scores.mean(), 'R2_std': scores.std()})

import pandas as pd
df_results = pd.DataFrame(results)
df_results

# 6. Comparación de Modelos <a id='sec6'></a>
**Intención:** identificar cuál funciona mejor.

Nota: Tener una tabla clara que diga qué modelo funcionó mejor y por cuánto.

- Comparar MAE, RMSE, R² en validación
- Gráficos comparativos

In [ ]:
df_results.sort_values(by='R2_mean', ascending=False)

# 7. Curvas de Aprendizaje <a id='sec7'></a>
**Intención:** detectar underfitting/overfitting. Puede ser solo para los 2 top.

In [ ]:
# from sklearn.model_selection import learning_curve

# 8. Análisis de Predicciones <a id='sec8'></a>
**Intención:** analizar calidad de predicciones.

- Scatter plot Predicho vs Real
- Distribución de errores
- Sesgos por rango

In [ ]:
# ejemplo de scatter real vs predicho

# 9. Selección de Mejores Modelos <a id='sec9'></a>
**Intención:** elegir modelos para optimización.
Elegir los top 2–3 modelos que pasarán a 05_model_optimization.ipynb.
Criterios:
- Mayor R²
- Menor MAE
- Estabilidad (desviación estándar baja)
- Tiempo de entrenamiento razonable

# 10. Resumen Ejecutivo <a id='sec10'></a>
**Intención:** documentar resultados y decisiones finales.
- Qué modelos se probaron
- Cuáles funcionaron mejor y por qué
- Qué problemas se identificaron
- Qué modelos pasan a optimización